In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.unit_proccessing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyod.models.ecod import ECOD
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.filter import FILTER

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = UnitDataProcessing(config)

In [ ]:
df_unit = features_class.df_unit
self = features_class
feature_name = 'f__pause_duration'
df = df_unit[(~pd.isnull(df_unit[feature_name]))].copy()
df[feature_name] = df[feature_name].astype(float)
df[feature_name] = round(df[feature_name]/(3600*2),0)
df[feature_name] = df[feature_name].apply(lambda x: round(x/12)*12 if x>12 else x)
#.apply(transform_duartion)#/df_unit[~pd.isnull(df_unit[feature_name])]['f__number_answered']
#/df_unit[~pd.isnull(df_unit[feature_name])]['f__number_answered']

|## Plot the distribution of answer time set

In [ ]:
df = self.get_df_pause()
df[feature_name] = round(df[feature_name]/(3600),0)
df[feature_name] = df[feature_name].apply(lambda x: round(x/24)*24 if x>24 else x)

In [ ]:
df[feature_name].hist()

## USE ECOD algorithm that makes use of cumulative function and is non-parametric for detecting anomalies in answer time set

In [ ]:
from pyod.models.mad import MAD
from  pyod.models.iforest import IForest #NO
from  pyod.models.gmm import GMM #NO
from  pyod.models.copod import COPOD#NO
from  pyod.models.qmcd import QMCD # NO
from  pyod.models.cof import COF
from  pyod.models.cblof import CBLOF#NO
from  pyod.models.hbos import HBOS#NO
from  pyod.models.inne import INNE #NO
from  pyod.models.lscp import LSCP#NO
from  pyod.models.lmdd import LMDD #NO
from  pyod.models.lof import LOF #NO
model = COF(contamination=0.11)#FILTER( random_state=42))
#model = ECOD(contamination=FILTER(method='medfilt', random_state=42))
#model = MAD(threshold=2.5)
#model.fit(df[df['f__time_changed']>=0][[feature_name]])
model.fit(df[[feature_name]])

In [ ]:
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.comb import COMB


from pythresh.thresholds.yj import YJ
decision_scores = model.decision_scores_  # raw outlier scores on the train data

# get outlier labels
#thres = HIST()
#thres = DSN()
thres = FILTER( random_state=42)
labels = thres.eval(decision_scores)
thres.thresh_

In [ ]:
df['anomaly'] = model.predict(df[[feature_name]])

In [ ]:
df['anomaly'].value_counts(), df['anomaly'].value_counts()/df['anomaly'].count()

In [ ]:
df[df['anomaly']==0][feature_name].min(), df[df['anomaly']==0][feature_name].max()

In [ ]:
#df.loc[ df[feature_name]<df[df['anomaly']==0][feature_name].max(), 'anomaly'] =0
df['anomaly'].value_counts()

In [ ]:
c=12*31#48*5
data_true = df[(df['anomaly']==0)&(df[feature_name]<c)][feature_name]/12
data_false = df[(df['anomaly']==1)&(df[feature_name]<c)][feature_name]/12

# Determine the bin edges based on the entire dataset
bins = np.histogram_bin_edges(df[(df[feature_name]<c)][feature_name]/12, bins=48)


plt.hist(data_true, bins=bins, alpha=0.5, color='blue', label='True')
plt.hist(data_false, bins=bins, alpha=0.5, color='red', label='False')
plt.show()

In [ ]:
sns.boxplot(df,y=feature_name, x='anomaly')
plt.show()

In [ ]:
bins = np.histogram_bin_edges(df[df[feature_name]<df[df['anomaly']==0][feature_name].max()][feature_name], bins=48)
plt.hist(df[df[feature_name]<df[df['anomaly']==0][feature_name].max()][feature_name], bins=bins, alpha=0.5, color='blue', label='True')
plt.show()

In [ ]:
df.groupby('survey_version').anomaly.value_counts()